In [1]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.preprocessing import LabelEncoder
import pandas as pd
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [2]:
train.columns

Index(['ID', '월', '요일', '시간', '소관경찰서', '소관지역', '사건발생거리', '강수량(mm)', '강설량(mm)',
       '적설량(cm)', '풍향', '안개', '짙은안개', '번개', '진눈깨비', '서리', '연기/연무', '눈날림',
       '범죄발생지', 'TARGET'],
      dtype='object')

In [4]:
train = train.drop(['월','시간','강수량(mm)','강설량(mm)','적설량(cm)','풍향','짙은안개','진눈깨비','서리','눈날림'],axis=1)
test = test.drop(['월','시간','강수량(mm)','강설량(mm)','적설량(cm)','풍향','짙은안개','진눈깨비','서리','눈날림'],axis=1)

In [5]:
train.TARGET.value_counts()
# [0 : 강도, 1: 절도, 2: 상해]

0    36453
1    25397
2    22556
Name: TARGET, dtype: int64

In [6]:
# tar = ['공원', '백화점', '병원', '식당', '약국', '은행', '인도', '주거지', '주유소', '주차장', '차도', '편의점', '학교', '호텔/모텔']
# import matplotlib.pyplot as plt
# for i in tar:
#     plt.figure(figsize=(10,8))
#     plt.bar(['강도','절도','상해'],[i for i in train[train['범죄발생지']==i]['TARGET'].value_counts()])
#     plt.title(f'{i}')
#     plt.show()

In [7]:
tp = []
for i in train['요일']:
    if i not in ['토요일','일요일']:
        tp.append(0)
    else:
        tp.append(1)
train['주말'] = tp
tp = []
for i in test['요일']:
    if i not in ['토요일','일요일']:
        tp.append(0)
    else:
        tp.append(1)
test['주말'] = tp
train.drop('요일',axis=1,inplace=True)
test.drop('요일',axis=1,inplace=True)

['공원', '백화점', '병원', '식당', '약국', '은행', '인도', '주거지', '주유소', '주차장', '차도', '편의점', '학교', '호텔/모텔']

In [8]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(77) # Seed 고정

In [9]:
x_train = train.drop(['ID'], axis = 1)
# y_train = train['TARGET']

x_test = test.drop(['ID'], axis = 1)

In [11]:
ordinal_features = ['범죄발생지']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(x_train[feature])
    x_train[feature] = le.transform(x_train[feature])

    # x_train데이터에서 존재하지 않았던 값이 x_test 데이터에 존재할 수도 있습니다.
    # 따라서 x_test 데이터를 바로 변형시키지 않고 고윳값을 확인후 x_test 데이터를 변환합니다.
    for label in np.unique(x_test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    x_test[feature] = le.transform(x_test[feature])

In [12]:
from pycaret import classification
classification.setup(data=x_train,
                     target='TARGET',
                     fold_strategy='kfold',
                     train_size=0.9,
                     session_id=77,
                     remove_outliers=True,
                     outliers_method='iforest')

,Description,Value
0,Session id,77
1,Target,TARGET
2,Target type,Multiclass
3,Original data shape,"(84406, 9)"
4,Transformed data shape,"(80607, 9)"
5,Transformed train set shape,"(72166, 9)"
6,Transformed test set shape,"(8441, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


In [13]:
best = classification.compare_models(n_select=5,fold=5,sort='F1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.5473,0.6996,0.5473,0.5441,0.5371,0.2859,0.2915,3.0240
xgboost,Extreme Gradient Boosting,0.5456,0.6969,0.5456,0.5415,0.5356,0.2835,0.2887,10.6920
gbc,Gradient Boosting Classifier,0.5443,0.6958,0.5443,0.5421,0.5337,0.2810,0.2870,16.8040
catboost,CatBoost Classifier,0.5434,0.6951,0.5434,0.5388,0.5334,0.2801,0.2852,20.3600
ada,Ada Boost Classifier,0.5401,0.6857,0.5401,0.5373,0.5309,0.2770,0.2817,3.1780
nb,Naive Bayes,0.4901,0.6351,0.4901,0.4818,0.4823,0.2027,0.2046,1.7720
knn,K Neighbors Classifier,0.4913,0.6370,0.4913,0.4795,0.4765,0.1948,0.1992,2.9480
qda,Quadratic Discriminant Analysis,0.4900,0.6378,0.4900,0.4783,0.4741,0.1914,0.1966,1.7700
rf,Random Forest Classifier,0.4671,0.6349,0.4671,0.4638,0.4651,0.1773,0.1775,7.6880
et,Extra Trees Classifier,0.4564,0.6134,0.4564,0.4538,0.4549,0.1617,0.1618,6.5480


Processing:   0%|          | 0/73 [00:00<?, ?it/s]

In [18]:
probas = []
for model in best:
    probas.append(model.predict_proba(x_test))
a,b,c,d,e = 0.6,0.2,0,0.2,0
probas = np.array(probas)
pred = [np.argmax(i) for i in a*probas[0]+b*probas[1]+c*probas[2]+d*probas[3]] # np.array(probas).mean(axis=0)

In [19]:
sub = pd.read_csv('./sample_submission.csv')
sub.set_index('ID',inplace=True)
sub['TARGET'] = pred
sub.to_csv('./answer.csv')